In [184]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import seaborn as sns
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import mlab
from matplotlib.ticker import FuncFormatter
matplotlib.style.use('ggplot')
import sqlite3 
import warnings
warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None


In [185]:
weeks = [210918, 210911, 210904, 210828,210821,210814,210807,210731,210724,210717,210710,210703,210626,210619]
url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'
df_list = []
for week in weeks:
    df_list.append(pd.read_csv(url.format(week)))
mta = pd.concat(df_list)


First basics Questions:
What is the busiest station ?

The station belongs to who (BMT, IRT, or IND)?

Which days is the most busiest?

In [208]:
mta


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,00:00:00,REGULAR,7633126,2611933
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,04:00:00,REGULAR,7633141,2611934
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,08:00:00,REGULAR,7633152,2611953
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,12:00:00,REGULAR,7633203,2611997
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,16:00:00,REGULAR,7633308,2612026
...,...,...,...,...,...,...,...,...,...,...,...
209255,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/18/2021,05:00:00,REGULAR,5554,584
209256,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/18/2021,09:00:00,REGULAR,5554,584
209257,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/18/2021,13:00:00,REGULAR,5554,584
209258,TRAM2,R469,00-05-01,RIT-ROOSEVELT,R,RIT,06/18/2021,17:00:00,REGULAR,5554,584


The New York subway MTA turnstile data is a series of data files containing cumulative number of entries and exits by station, turnstile, date and time. Data files are produced weekly, data records are collected typically every 4 hours with some exceptions.


In this analysis we use data from the last 3 Months of 2021. Data size is over 5 million.

In [209]:
mta.columns =['C/A','UNIT','SCP','STATION','LINENAME','DIVISION','DATE','TIME','DESC','ENTRIES','EXITS']

In [210]:
mta['DATETIME'] = pd.to_datetime(mta.DATE + ' ' + mta.TIME, format='%m/%d/%Y  %H:%M:%S')
mta['TURNSTILE'] = mta['C/A'] + '-' + mta['UNIT'] + '-' + mta['SCP']

mta = mta[(mta.DATETIME >= '01-01-18 00:00:00') & (mta.DATETIME <'09-30-21 00:00:00')]

In [211]:
mta.head(10)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,TURNSTILE
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,00:00:00,REGULAR,7633126,2611933,2021-09-11 00:00:00,A002-R051-02-00-00
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,04:00:00,REGULAR,7633141,2611934,2021-09-11 04:00:00,A002-R051-02-00-00
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,08:00:00,REGULAR,7633152,2611953,2021-09-11 08:00:00,A002-R051-02-00-00
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,12:00:00,REGULAR,7633203,2611997,2021-09-11 12:00:00,A002-R051-02-00-00
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,16:00:00,REGULAR,7633308,2612026,2021-09-11 16:00:00,A002-R051-02-00-00
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/11/2021,20:00:00,REGULAR,7633457,2612050,2021-09-11 20:00:00,A002-R051-02-00-00
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/12/2021,00:00:00,REGULAR,7633508,2612065,2021-09-12 00:00:00,A002-R051-02-00-00
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/12/2021,04:00:00,REGULAR,7633518,2612070,2021-09-12 04:00:00,A002-R051-02-00-00
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/12/2021,08:00:00,REGULAR,7633528,2612075,2021-09-12 08:00:00,A002-R051-02-00-00
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,09/12/2021,12:00:00,REGULAR,7633565,2612106,2021-09-12 12:00:00,A002-R051-02-00-00


1. Sanity Check
1.1 Original data

First we look for potential abnormal entries and exits values. Since entries and exits are cumulative values, quantiles do not mean anything, but there should not be any negative values. We expect entries to be larger than exits in general, because New York subway stations commonly have emergency exits, which do not collect exit records. Exits are only collected when a passenger exit through a turnstile.

In [212]:
print('Descriptions of entries:')
print(mta['ENTRIES'].describe())
print('')
print('Descriptions of exits:')
print(mta['EXITS'].describe())

Descriptions of entries:
count    2.932689e+06
mean     4.154567e+07
std      2.182202e+08
min      0.000000e+00
25%      2.191160e+05
50%      1.400465e+06
75%      6.001544e+06
max      2.147412e+09
Name: ENTRIES, dtype: float64

Descriptions of exits:
count    2.932689e+06
mean     3.321487e+07
std      1.916933e+08
min      0.000000e+00
25%      1.016850e+05
50%      8.535230e+05
75%      3.970447e+06
max      2.133797e+09
Name: EXITS, dtype: float64


1.2 Use differenced data instead

Then we calculate the differences between every two collection timestamps and look for abnormal entries/exits per time interval. NaN values are generated for the very first data record for each unique turnstile during differencing.

In [213]:
mta_sorted = mta.sort_values(['TURNSTILE', 'DATETIME'])
mta_sorted = mta_sorted.reset_index(drop = True)

turnstile_grouped = mta_sorted.groupby(['TURNSTILE'])

mta_sorted['ENTRIES_diff'] = turnstile_grouped['ENTRIES'].transform(pd.Series.diff)
mta_sorted['EXITS_diff'] = turnstile_grouped['EXITS'].transform(pd.Series.diff)

mta_sorted.head(100000)

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,TURNSTILE,ENTRIES_diff,EXITS_diff
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,00:00:00,REGULAR,7585482,2593043,2021-06-12 00:00:00,A002-R051-02-00-00,NaN,NaN
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,04:00:00,REGULAR,7585492,2593043,2021-06-12 04:00:00,A002-R051-02-00-00,10.0,0.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,08:00:00,REGULAR,7585498,2593050,2021-06-12 08:00:00,A002-R051-02-00-00,6.0,7.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,12:00:00,REGULAR,7585546,2593066,2021-06-12 12:00:00,A002-R051-02-00-00,48.0,16.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,16:00:00,REGULAR,7585641,2593068,2021-06-12 16:00:00,A002-R051-02-00-00,95.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,A043,R462,00-00-01,CANAL ST,JNQRZ6W,BMT,09/17/2021,21:00:00,REGULAR,355833434,441361735,2021-09-17 21:00:00,A043-R462-00-00-01,230.0,346.0
99996,A043,R462,00-00-02,CANAL ST,JNQRZ6W,BMT,06/12/2021,01:00:00,REGULAR,14564372,13978955,2021-06-12 01:00:00,A043-R462-00-00-02,NaN,NaN
99997,A043,R462,00-00-02,CANAL ST,JNQRZ6W,BMT,06/12/2021,05:00:00,REGULAR,14564378,13978959,2021-06-12 05:00:00,A043-R462-00-00-02,6.0,4.0
99998,A043,R462,00-00-02,CANAL ST,JNQRZ6W,BMT,06/12/2021,09:00:00,REGULAR,14564389,13978969,2021-06-12 09:00:00,A043-R462-00-00-02,11.0,10.0


In [214]:
del mta

Sanity check of entries_diff and exits_diff; number of observations with NaN should equal the number of unique turnstiles. entries_diff and exits_diff should always be positive since cumulative values are supposed to increase or at least stay even.

In [215]:
# check distribution of entries_diff and exits_diff
print('Descriptions of entries_diff:')
print(mta_sorted['ENTRIES_diff'].describe())
print('')
print('Descriptions of exits_diff:')
print(mta_sorted['EXITS_diff'].describe())

Descriptions of entries_diff:
count    2.927659e+06
mean    -4.606856e+02
std      2.086780e+06
min     -1.383110e+09
25%      3.000000e+00
50%      2.600000e+01
75%      8.300000e+01
max      1.889997e+09
Name: ENTRIES_diff, dtype: float64

Descriptions of exits_diff:
count    2.927659e+06
mean    -8.471182e+01
std      3.121646e+06
min     -2.133741e+09
25%      5.000000e+00
50%      3.000000e+01
75%      9.000000e+01
max      2.133741e+09
Name: EXITS_diff, dtype: float64


In [216]:
print('Number of negative entries_diff: %d' %len(mta_sorted['ENTRIES_diff'][mta_sorted['ENTRIES_diff'] < 0]))
print('Number of negative exits_diff: %d' %len(mta_sorted['EXITS_diff'][mta_sorted['EXITS_diff'] < 0]))
print('Number of unqiue turnstiles: %d' %len(mta_sorted['TURNSTILE'].unique()))
print('Number of NaN rows: %d' %len(mta_sorted[mta_sorted['ENTRIES_diff'].isnull()]))

Number of negative entries_diff: 25620
Number of negative exits_diff: 17415
Number of unqiue turnstiles: 5030
Number of NaN rows: 5030


The max values of entries_diff and exits_diff are more than a million times larger than their 75th percentiles, which is apparently abnormal. Minimum values are negative, which is also abnormal. We will set these outliers as 0. After taking a look at observations with negative _entriesdiffs, we find out that of them have DESC == 'DOOR CLOSE'. It seems that the entry and exit counts might be reset when experiencing door close. It would be inappropriate to infer the correct values for these observations, therefore we will set them as 0. We also set NAs as 0 since they are the first data record for each turnstile.

In [217]:
mta_sorted['ENTRIES_diff'] = mta_sorted['ENTRIES_diff'].fillna(0)
mta_sorted['EXITS_diff'] = mta_sorted['EXITS_diff'].fillna(0)

mta_sorted['ENTRIES_diff'][mta_sorted['ENTRIES_diff'] < 0] = 0 
mta_sorted['EXITS_diff'][mta_sorted['EXITS_diff'] < 0] = 0 


In [218]:
mta_sorted.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,TURNSTILE,ENTRIES_diff,EXITS_diff
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,00:00:00,REGULAR,7585482,2593043,2021-06-12 00:00:00,A002-R051-02-00-00,0.0,0.0
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,04:00:00,REGULAR,7585492,2593043,2021-06-12 04:00:00,A002-R051-02-00-00,10.0,0.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,08:00:00,REGULAR,7585498,2593050,2021-06-12 08:00:00,A002-R051-02-00-00,6.0,7.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,12:00:00,REGULAR,7585546,2593066,2021-06-12 12:00:00,A002-R051-02-00-00,48.0,16.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,06/12/2021,16:00:00,REGULAR,7585641,2593068,2021-06-12 16:00:00,A002-R051-02-00-00,95.0,2.0
